In [1]:
#!pip install scikit-learn scipy matplotlib numpy
import sys
sys.path.insert(0, "..")

from base import *
from util import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import re
import numpy as np

from tensorflow.keras.utils import to_categorical

from fitter import Fitter, get_common_distributions, get_distributions
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.datasets import make_friedman3, make_s_curve
from sklearn.linear_model import BayesianRidge, ElasticNet
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.neural_network import MLPClassifier

#!pip install imbalanced-learn
#!pip install lightgbm
#!pip install wittgenstein --user
#!pip install explainerdashboard --user

from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


# for Box-Cox Transformation
from scipy import stats

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px

from tqdm import tqdm
import math

from sklearn.datasets import make_blobs
from matplotlib import pyplot
from pandas import DataFrame


In [2]:
#Collect data from sensors (i.e., time-series data) installed in the manufacturing system 
ai4i2020_encoded_balanced=pd.read_csv(r'G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\raw_data\\ai4i2020_encoded_balanced.csv')
ai4i2020_encoded_balanced.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,0,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,0,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,0,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,0,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [3]:
# Run in local
# Fold index
fold = 1

original_X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
original_X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

X_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_train = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")

In [4]:
ai4i2020_encoded_balanced = ai4i2020_encoded_balanced.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

## Hyper-parameter Tuning in FCMB Classifier

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from base.classifier import BaseFcmClassifier
from base.losses import MSE, LogLoss, SoftmaxCrossEntropy
from base.model import Model, AdditiveModel, Sigmoid, Tanh, Ident, Relu
from base.binary_classifier import FcmBinaryClassifier
import tensorflow as tf
import sklearn.preprocessing
import numpy as np

#we are training our model with the batch size ranging from 8 to 2048 with each batch size twice the size of the previous batch size.
fcmb_param_grid = {'act': [None],
               'activation': ['sigmoid'],
               'activation_m': [1],
               'depth': [2, 3, 5],
               'epochs': np.arange(20, 100,10),
               'batch_size': 2**np.arange(3,12), 
               'buffer_size': [1000],
               'training_loss': ['logloss'],
               'optimizer': ['rmsprop'],
               'learning_rate': np.arange(0.01, 1, 0.099),
               'model_type': ['basic'],
               'init_method': [None],
               'init_y_method': ['uniform'],
               'dropout_p': [0.0],
               'random_state': [None]}

grid = ParameterGrid(fcmb_param_grid)


X = X_train.to_numpy()
y = y_train.to_numpy()

acc_list = []
auc_list = []
kappa_list = []

twf_list = []
hdf_list = []
pwf_list = []
osf_list = []
av_succ = []

comb_list = []

for comb in range(0,len(grid)):
    
    #Build the FCMB Classifier
    fcmb = FcmBinaryClassifier()
    display("Combination " + str(comb))
    fcmb.__init__(**grid[comb])
    fcmb.fit(X,y)
    
    #Predictions for the test set
    y_test_pred_fcmb = fcmb.predict(X_test)
    
    #Evaluation metrics
    accuracy_fcmb = accuracy_score(y_test, y_test_pred_fcmb)
    auc_fcmb = metrics.roc_auc_score(y_test, y_test_pred_fcmb)
    cohen_score_fcmb = cohen_kappa_score(y_test, y_test_pred_fcmb)

    
    
    #FCMB trajectory
    init_y_test = np.ones((X_test.shape[0], fcmb.base_fcm.model.dim - X_test.shape[1])) * .5
    trj_fcmb_test = fcmb.base_fcm.model.trajectory(X_test, init_y_test)
    
    
    local_feature_importance_fcmb = pd.DataFrame(trj_fcmb_test[fcmb.depth], columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'])
    local_feature_importance_fcmb.drop('Machinefailure', inplace=True, axis=1)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_fcmb[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_list.append(accuracy_fcmb)
        auc_list.append(auc_fcmb)
        kappa_list.append(cohen_score_fcmb)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)
        
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_fcmb.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_fcmb.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_fcmb.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;
    
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_list.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
        else:
            twf_list.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_list.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
        else:
            hdf_list.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_list.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
        else:
            pwf_list.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_list.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
        else:
            osf_list.append(correct_explanations_OSF)
            osf_success = 0
    
        average_success = (twf_success+hdf_success+pwf_success+osf_success)/4
    
        av_succ.append(average_success)
        comb_list.append(comb)
    
    
    
result_df = pd.DataFrame({'Accuracy': acc_list, 'AUC': auc_list, 'Kappa':kappa_list, 'TWF':twf_list, 'HDF':hdf_list, 'PWF':pwf_list, 'OSF':osf_list, 'Combinations':comb_list})
final_df = result_df.sort_values(by=['AUC'], ascending=False)
final_df = final_df.reset_index(drop=True)
best_estimator_params = grid[final_df.loc[0,"Combinations"]]
display(best_estimator_params) 

'Combination 0'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 8, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 20, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.01, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios Tirovolas\Papers_Jupyter_Notebook_Codes\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study-main\fcm_classifier_transformer-main\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 430.38666462898254 sec



'Combination 1'

...

C:\Users\Marios Tirovolas\Papers_Jupyter_Notebook_Codes\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study-main\fcm_classifier_transformer-main\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')


Learning time 1031.6294808387756 sec



'Combination 36'

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
{'act': None, 'activation': 'sigmoid', 'activation_m': 1, 'batch_size': 8, 'buffer_size': 1000, 'depth': 2, 'dropout_p': 0.0, 'epochs': 50, 'init_method': None, 'init_y_method': 'uniform', 'learning_rate': 0.6040000000000001, 'model_type': 'basic', 'optimizer': 'rmsprop', 'random_state': None, 'training_hook': [], 'training_loss': 'logloss'}
-----------------------------------------


C:\Users\Marios Tirovolas\Papers_Jupyter_Notebook_Codes\Data-driven-Causal-Discovery-for-Constructing-Fuzzy-Cognitive-Maps-An-Industrial-Case-Study-main\fcm_classifier_transformer-main\base\classifier.py:143: UserWarning: 2018.12.16 The instruction  'loss = loss/self.batch_size' was commented out. Due to increased gradient values check the optimizer learning rate 
  warnings.warn('2018.12.16 The instruction  \'loss = loss/self.batch_size\' was commented out. Due to increased gradient values check the optimizer learning rate ')
